In [1]:
import sys
import os
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('../utils_meinhold')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('../utils_zonca')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca\\pointing')
sys.path.append('../utils_zonca/pointing')
#sys.path.append('D:\\software_git_repos\\greenpol')
sys.path.append('../')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\')
sys.path.append('../telescope_control')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\VtoT\\')
sys.path.append('../VtoT')
import realtime_gp as rt
import numpy as np
import datetime  
import h5py
import cofe_util as cu
import convert

from functions import *
from scipy.optimize import curve_fit


%pylab

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(f,stats.st_size)? (realtime_gp.py, line 84)

In [2]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
def get_pointing_files(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/Polaris/polaris_data/pointing_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    return filelist

In [4]:
def get_pointing_files(filelist=None):
    if filelist == None:
        flist = []
        root = Tkinter.Tk()
        files = list(tkFileDialog.askopenfilenames( \
            initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/pointing_data/',
            parent=root, title='Choose a set of files'))
        flist.append(files)
        root.destroy()
        filelist = [y for x in flist for y in x]
    filelist.sort()

    return filelist

In [5]:
def get_h5_pointing(filelist,startrev=None, stoprev=None,angles_in_ints=False,azel_era=3):
    
    errlimit=0.1
    if angles_in_ints==True:
        errlimit=10
    hpointing=[]
    filelist.sort()
    for f in filelist:
        if f[79:88] != 'corrected':
            slimit = 22144
        else:
            slimit = 11512
        stats=os.stat(f)
        if stats.st_size<slimit:
            print f,stats.st_size
        if stats.st_size >= slimit:
            h=h5py.File(f)
            hh=h['data']
            hpointing.append(hh[hh['Time']>=hh['Time'][0]])
            #print hpointinng
            h.close()

    hpointing = np.concatenate(hpointing)
    #cut out blank lines from unfilled files

    hrevlist,inds=np.unique(hpointing['Time'],return_index=True)

    psize = len(hpointing['Time'][inds])

    htmeans=hpointing['Horn Top'][inds]
    spmeans=hpointing['Shield Plate'][inds]
    coolermeans = hpointing['Cooler'][inds]
    transitionmeans = hpointing['Transition'][inds]
    cpmeans = hpointing['Copper Plate'][inds]
    ampmeans = hpointing['Amplifier'][inds]

    return {'Horn Top':htmeans,'Shield Plate':spmeans,'gpstime':hrevlist, 'Cooler':coolermeans
            , 'Transition':transitionmeans, 'Copper Plate':cpmeans, 'Amplifier':ampmeans}

In [6]:
def read_some_data(datadict,datalabel,datatype='demod', filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        if datatype=='demod':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='D://software_git_repos/Polaris/polaris_data/demod_data/',parent=root,title='Choose a set of files'))
        if datatype=='raw':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='D://software_git_repos/Polaris/polaris_data/data/',parent=root,title='Choose a set of files'))            
        root.destroy()
    filelist.sort()
    
    if datatype=='demod':
        dlist=[]
        for f in filelist:
            hf=h5py.File(f)
            dlist.append(hf['demod_data'])
        d=concatenate(dlist)
        hf.close() 
        
    if datatype=='raw':
        d=rt.demod.read_raw(filelist)
    datadict[datalabel]=d
    return

In [7]:
def asd_from_data(data,minfreq=1.):
    #run nps on all the channels for each of the loaded data sets, return asd dictionary
    
    asd_dict = {}
    for datset in data.keys():
        asd_dict[datset]={}
        d=data[datset][:100000]
        samprate=1000./float(data['time'][10]-data['time'][9])
        
        freq,psd=cu.nps(d,samprate,minfreq=minfreq)
        asd_dict[datset] =sqrt(psd)
        asd_dict['freq']=freq
                    
    return asd_dict            

In [8]:
def plot_asd_dict(asd_dict, datalist=None):
    #function to give a choice of preloaded datasets from dictionary data_dict
    plt.figure()
    if datalist==None:
        datalist=asd_dict.keys()

    for dname in datalist:
        if dname == 'freq' or dname == 'time':# or dname == '5. Copper Plate' or dname == '6. Amplifier':
            continue
        asd=asd_dict[dname]
        label = dname
        #if dname == '4. Transition':
        #    label = '1st stage'
        #if dname == '3. Cooler':
        #    label = '2nd stage'
        plt.plot(asd_dict['freq'],asd, label=label,lw=2)

    plt.legend()
    plt.xlabel('Frequency [Hz]')
    plt.ylabel(r'ASD [$\frac{K}{\sqrt{Hz}}$]')

In [9]:
flp = {}
pp = {}

In [10]:
#check to see if you horn top and shield plate switched in previous tests
#by seeing if there is a consistent difference in temperature when warm

In [11]:
VT = {'time': 'c', '1. Horn Top':'i','2. Shield Plate':'c', '3. Cooler':'h'
            , '4. Transition':'l', '5. Copper Plate':'k', '6. Amplifier':'e'}

In [12]:
numdays = 1

for i in range(1, numdays+1):
    print i

    flp[i] = get_pointing_files()
    pp[i] = get_h5_pointing(flp[i])

1


C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:17: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


In [13]:
sensors = {}
for i in range(1, numdays+1):
    sensors[i] = {'time': pp[i]['gpstime'], '1. Horn Top':pp[i]['Horn Top'],'2. Shield Plate':pp[i]['Shield Plate'], '3. Cooler':pp[i]['Cooler']
            , '4. Transition':pp[i]['Transition'], '5. Copper Plate':pp[i]['Copper Plate'], '6. Amplifier':pp[i]['Amplifier']}

In [14]:
#asd_dict = asd_from_data(sensors, minfreq=0.1)

In [15]:
#plot_asd_dict(asd_dict)

In [16]:
i = 1
#plt.figure()
for s in sensors[i]:
    
    #for i in range(1, numdays+1):
    time = (sensors[i]['time']-sensors[i]['time'][0])/3600. #+ (12+(29/60.)+(6/3600.))
    #if s != '3. Cooler':
    #    continue  
    plt.figure()
        #if i == 1: 
        #    plt.plot(time, sensors[i][s],label = '%s, 2/2021' % s)
        #else:
        #    plt.plot(time, sensors[i][s],label = '%s, 3/2022' % s)
    
    temp = convert.convert(sensors[i][s], VT[s])
    plt.plot(time, temp,label = s+', day'+str(i))
    plt.xlabel('time (hours)')
    plt.ylabel('temperature (K)')
    plt.title(s)

    plt.legend()

In [50]:
time = (pp[i]['gpstime'] - pp[i]['gpstime'][0])/3600. #+ (17 + 32./60 + 15/3600.)

fig, axs = plt.subplots(2, 3)
s = "1. Horn Top"
axs[0, 0].plot(time, sensors[s])
axs[0, 0].set_title(s)
axs[0,0].set_ylabel('Temperature (K)')
s = "2. Shield Plate"
axs[1, 0].plot(time, sensors[s])
axs[1, 0].set_title(s)
axs[1,0].set_ylabel('Temperature (K)')
axs[1,0].set_xlabel('time (hours)')
s = "3. Cooler"
axs[0, 1].plot(time, sensors[s])
axs[0, 1].set_title("2nd Stage")
fig.tight_layout()
s = "4. Transition"
axs[1, 1].plot(time, sensors[s])
axs[1, 1].set_title("Transition")
axs[1,1].set_xlabel('time (hours)')
s = "5. Copper Plate"
axs[0, 2].plot(time, sensors[s])
axs[0, 2].set_title("Copper Plate")
fig.tight_layout()
s = "6. Amplifier"
axs[1, 2].plot(time, sensors[s])
axs[1, 2].set_title("Amplifier")
axs[1,2].set_xlabel('time (hours)')
fig.tight_layout()

In [ ]:
#plt.figure()
s = 'Amplifier'
plt.plot(time, sensors[s] - sensors[s].max(), label = s)
plt.xlabel('time (hours)')
plt.ylabel('temperature (V)')
plt.title(s)

plt.legend()

In [23]:
i = 1
stage2 = pp[i]['Cooler']
time = (pp[i]['gpstime'] - pp[i]['gpstime'][0])/3600. + (13 +32./60 + 54/3600.)
stage1 = pp[i]['Transition']

In [25]:
plt.figure()
plt.plot(time, stage1)

In [76]:
# create figure and axis objects with subplots()
fig,ax = plt.subplots()
# make a plot
ax.plot(time, stage2, color="black")
# set x-axis label
ax.set_xlabel("time (hours)",fontsize=14)
# set y-axis label
ax.set_ylabel("stage 2 Temperature (K)",color="black",fontsize=14)


# twin object for two different y-axis on the sample plot
ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(time, stage1,color="blue")
ax2.set_ylabel("stage 1 Temperature(K)",color="blue",fontsize=14)
plt.title('Dewar Orientation tests, 90 deg = facing up')
plt.show()

In [128]:
print get_stats(stage1, 816, 1071)

(72.59031382850795, 1.8105411685295387)


In [72]:
def get_stats(data, lower_bound, upper_bound):
    
    mean = data[lower_bound:upper_bound].mean()
    std = data[lower_bound:upper_bound].std()
    
    return mean, std

In [160]:
temp1, std1 = get_stats(stage2, 0, 103)
temp2, std2 = get_stats(stage2, 1082, 1131)
temp3, std3 = get_stats(stage2, 1169, 1536)
temp4, std4 = get_stats(stage2, 1550, 1621)
temp5, std5 = get_stats(stage2, 1634, 2020)
temp6, std6 = get_stats(stage2, 2091, 2406)
temp7, std7 = get_stats(stage2, 2470, 2798)
temp8, std8 = get_stats(stage2, 2905, 3185)
temp9, std9 = get_stats(stage2, 3240, 3627)

print temp1, temp2, temp3, temp4, temp5, temp6, temp7, temp8, temp9
#print std1, std2, std3, std4, std5, std6, std7, std8, std9

13.272223364494842 13.054346663743624 13.196706599646626 14.224020262296436 14.814216782201267 16.21653537853423 17.315617422410927 19.44715432739259 22.08969060897631


In [103]:
plt.errorbar??

In [61]:
inputP = [0, 0.5, 1., 1.5, 2., 3., 4., 5., 6.]
temp = [13, temp3, 13.7, temp4, temp5, temp6, temp7, temp8, temp9]
std = [std1, std2, std3, std4, std5, std6, std7, std8, std9]



NameError: name 'temp3' is not defined

In [62]:
plt.figure()
plt.errorbar(inputP, temp, yerr=std, linewidth=2, capsize=3, label='real case')
plt.plot(inputP_ideal, temp_ideal, linewidth=2, label='ideal case')
plt.xlabel('2nd stage heat load (watts)')
plt.ylabel('2nd stage temperature (K)')
plt.legend()

NameError: name 'temp' is not defined

In [87]:
def func(T1s, q, k):
    return q*x/k/l/w + T1s

In [108]:
#normal configuration, foil in front of windows
#temperatures for different configurations
T1s = np.array([37., 61, 54, 50, 243]) #Stage 1 temperature
Tsp = np.array([282., 152, 266, 259, 215]) #shield plate temperature
k = 401 # copper thermal conductivity W/(K m)
l = 0.1016 #meters (measured ~4 inch straps)
w = 0.00762 #meters (measured ~0.3 inch)
A = l*w
x = 0.0010668 #thickness in meters (measured ~0.042 inch with callipers)

#l2 = 0.1778
#w2 = 0.0254
#x2 = x
#l = 12*l + l2
#w = 12*w + w2
#x = 12*x

dT = Tsp - T1s
q = k*A*dT/x

print q/1000

[71.2978     26.48204    61.69442286 60.82138857 -8.14832   ]


In [ ]:
idx = np.argsort(dT)
dT = dT[idx]
q = q[idx]

popt, pcov = curve_fit(func, T1s, Tsp)

In [102]:
plt.figure()
plt.plot(dT, q/1000., '.', linewidth=2)
plt.xlabel(r'$\Delta T$ (K)')
plt.ylabel('Heat loss (kW)')

Text(0,0.5,'Heat loss (kW)')

In [77]:
plt.figure()
plt.plot(T1s, q/1000., 'bo', linewidth=2, label='calculated Q')
plt.plot(dT, func(dT, *popt), 'r-', label='fit: m=%.3f kW/K, Vz=%.3f kW' % tuple(popt))
plt.xlabel(r'$T_{stage 1}$ (K)')
plt.ylabel('Heat loss Q (kW)')
plt.legend()

In [78]:
print k*l*w/x

281.311047619


In [53]:
#temperatures across straps
T1s = np.array([56., 73., 102., 51., 53.]) #Stage 1 temperature
Tsp = np.array([160., 162.,169, 121., 135.]) #shield plate temperature
P = np.array([0., 10., 20., 0., 0.])
k = 401. # copper thermal conductivity W/(K m)
l = 0.1016 #meters (measured ~4 inch straps)
w = 0.00762 #meters (measured ~0.3 inch)
x = 0.0010668 #thickness in meters (measured ~0.042 inch with callipers)
A = w*x

dT = Tsp - T1s
q = 6*k*A*dT/l

print q

[20.02042224 17.13286134 12.89777202 13.4752842  15.78533292]


In [143]:
plt.figure()
plt.plot(P, q, 'ko')
plt.xlabel('input power (W)')
plt.ylabel('Thermal straps heat loss (W)')

Text(0,0.5,'Thermal straps heat loss (W)')

In [134]:
#spider
T2s = [32, 38.,49.]
Tcp = np.array([172., 176., 185.])
P = np.array([0., 10., 20.])
k = 401. # copper thermal conductivity W/(K m)
l = 0.10668 + 0.01905 #2nd number is radius of inner part of spider, rough approximation of total length
w = 0.0254
x = 0.005334
A = w*x
print A

dT = Tcp - T2sq = k*A*dT/l



print q*3 #3 arms



0.0001354836
[181.48530909 178.89266182 176.30001455]


In [13]:
#calculation for big  strap
#density 
m = 13.5
l = 12.192
p = 8.96

A = m/p/l

print A

#length measurement
w = 1.63
x = 0.155

A2 = w*x

print A2

#area scale for previous measurement
s = A/A2
print s

0.123580743813
0.25265
0.489138111274


In [14]:
m = 6.6
l = 10.16
A = m/p/l
print A

w = 1.27
x = 0.11

A2 = w*x

print A2

#area scale for previous measurement
s = A/A2
print s

0.0725007030371
0.1397
0.518974252234
